# - The numbers 電影爬蟲 -
<ul>
    <li>
        以下分成二階段式爬蟲，最後存成csv檔以供後續分析使用
    </li>
        <ul>
            <li>
                第一階段.根據the numbers網站獲取最賺錢之電影前5575名，並且爬取所有資料回來
            </li>
            <li>
                第二階段.根據第一階段產生的資料藉由程式轉換成IMDB的網頁並存成陣列提供給第三階段使用
            </li>
        </ul>
</ul>

# - 時間戳記 -

<ul>
    <li>
        9/1 - 開始撰寫爬蟲。
    </li>
    <li>
        9/4 - 第一階段完成，儲存檔案為movie_numbers.csv，著手第二階段。
    </li>
    <li>
        9/10 - 第二階段發現numbers的電影名稱跟IMDB名稱不同，故手動修改，並加入異常處理迴圈。
    </li>
    <li>
        9/15 - 電影名稱錯誤更改完成，尚未測試。
    </li>
    <li>
        9/15 - 第二階段測試後發現有些類型是TV movie，有可能因為類型不同引發AttributeError。
    </li>
    <li>
        9/17 - 第二階段完成，儲存檔案為imdb_movie_urls.csv。
    </li>
</ul>

## 第一階段

In [1]:
# import libs
import requests
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import urllib
import re
from bs4 import BeautifulSoup

In [2]:
mv = [] #存放the numbers爬取下來的所有電影數據
url_number = [] #存放the numbers頁數所有url後面的數字
for num in range(1,5600,100):
    url_number.append(num)
#url_number

In [3]:
url = 'https://www.the-numbers.com/movie/budgets/all' # define url for crawling
resp = requests.get(url) # GET request from url and parse via BeautifulSoup
resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
soup = BeautifulSoup(resp.text, 'lxml')
# 在觀察網頁之後，發現表格中每一列的文字都躲在 tr 標籤底下，並用 td 包起來。
all_rows = soup.table.find_all("tr") # 找出所有 tr 標籤，並存成 list
column_name_tag = all_rows[0] # 標題名稱的標籤就是 all_rows 的第一筆資料 
column_name = [text for text in column_name_tag.stripped_strings] # 把這些標題存成 column_name 這個變數
#column_name

In [4]:
#將陣列轉換為dataframe
for i,x in enumerate(url_number):
    resp = requests.get("https://www.the-numbers.com/movie/budgets/all/" + str(x))
    resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
    soup = BeautifulSoup(resp.text, 'lxml')
    table = soup.table.find_all("tr") # 找出所有 tr 標籤，並存成 list
    #print("開始爬取the-numbers第 ", x , "項")
   # print("https://www.the-numbers.com/movie/budgets/all/" + str(x))
    
    for z, row in enumerate(table): # 表格偶數欄為空值，故只爬取基數欄
        if z % 2 == 0:
            continue
        data_want = [s for s in row.stripped_strings]
        del data_want[0] #去除list第一個元素（dataframe會自動幫我們標記，故去除第一列的編號）
        mv.append(data_want)#新增到list裡面
        #print(data_want)

In [5]:
data=DataFrame(mv,columns = column_name)#將陣列轉換為dataframe
data.to_csv("movie_numbers.csv", index = False, encoding="utf-8")#儲存為csv檔
data.head()

,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,12/18/2009,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,5/20/2011,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,5/1/2015,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,408,064,754"
3,12/18/2015,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,058,601,220"
4,4/27/2018,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,014,285,135"


## 第二階段

In [6]:
imdb_ID_urls = [] #存放imdb單個電影（imdb_ID）的url
imdb_search_urls = [] #存放imdb查詢的url
movie = data['Movie'] #將電影名稱取出來供後面爬蟲使用

#透過np轉換成array在轉換成list
movie_name = np.array(movie)#np.ndarray()
movie_name_list = movie_name.tolist()#list

In [7]:
#因the numbers上面的電影名稱會因為音譯問題名稱跟imdb不符合，無法使用程式修改，故手動修改共＃個電影有誤
movie_name_list[0] = 'Afanda' # 片名錯誤（阿凡達），故手動修改
movie_name_list[3] = 'Star Wars: Episode VII - The Force Awakens' # 片名錯誤(星際大戰七)，故手動修改
movie_name_list[550] = 'Fantasia 2000' # 片名錯誤（幻想曲2000），故手動修改
movie_name_list[590] = 'Astérix & Obélix: Au service de sa Majesté' # 片名錯誤（高盧英雄英遊記），故手動修改
movie_name_list[728] = 'The Lorax' # 片名錯誤（羅雷司），故手動修改
movie_name_list[911] = 'Xi you ji zhi: Sun Wukong san da Baigu Jing' # 片名錯誤（西遊記之孫悟空三打白骨精），故手動修改
movie_name_list[926] = 'Che: Part One' # 片名錯誤（切·格瓦拉），故手動修改
movie_name_list[1920] = 'Xin hai ge ming' # 片名錯誤（辛亥革命），故手動修改
movie_name_list[2304] = 'Chai dan zhuan jia' # 片名錯誤（拆彈專家），故手動修改
movie_name_list[2603] = 'Kolchak: The Night Stalker' # 片名錯誤（Kolchak: The Night Stalker），故手動修改
movie_name_list[3031] = 'The Steal' # 片名錯誤（The Steal ），故手動修改
movie_name_list[3790] = 'Fei chang xing yun' # 片名錯誤（非常幸運 ），故手動修改
movie_name_list[3045] = 'Dungeons & Dragons: Wrath of the Dragon God' # 片名錯誤（魔龍傳奇II：仁者無敵 ），故手動修改
movie_name_list[3674] = 'Red Riding: The Year of Our Lord 1974' # 片名錯誤（血迷宫：1974 ），故手動修改
movie_name_list[3916] = 'Adventures in Appletown' # 片名錯誤（Adventures in Appletown ），故手動修改
movie_name_list[3917] = 'Stargate: The Ark of Truth' # 片名錯誤（Stargate: The Ark of Truth），故手動修改
movie_name_list[3992] = 'Dolphins and Whales 3D: Tribes of the Ocean' # 片名錯誤（Dolphins and Whales 3D: Tribes of the Ocean），故手動修改
movie_name_list[4021] = 'Ang-ma-reul bo-at-da' # 片名錯誤（我看見魔鬼），故手動修改
movie_name_list[4221] = 'Dead Like Me: Life After Death' # 片名錯誤（死神有約：死後的生活），故手動修改
movie_name_list[4296] = "R.L. Stine's Monsterville: Cabinet of Souls" # 片名錯誤（RL斯坦的怪獸村：靈魂櫃房），故手動修改
movie_name_list[4456] = "Batman: The Dark Knight Returns, Part 2" # 片名錯誤（蝙蝠俠：黑闇騎士歸來（下）），故手動修改
movie_name_list[4585] = "Never Back Down 2: The Beatdown" # 片名錯誤（地下拳擊社2），故手動修改
movie_name_list[4746] = "Son of Rambow" # 片名錯誤（Son of Rambow），故手動修改
movie_name_list[4956] = "The 41-Year-Old Virgin Who Knocked Up Sarah Marshall and Felt Superbad About It" # 片名錯誤（The 41-Year-Old Virgin Who Knocked Up Sarah Marshall and Felt Superbad About It），故手動修改
movie_name_list[5271] = "Feng kuang de shi tou" # 片名錯誤（瘋狂的石頭），故手動修改
movie_name_list[5294] = "The Little Ponderosa Zoo" # 片名錯誤（The Little Ponderosa Zoo），故手動修改
movie_name_list[5433] = "Chats perchés" # 片名錯誤（Chats perchés），故手動修改
movie_name_list[5438] = "Samantha: An American Girl Holiday" # 片名錯誤（Samantha: An American Girl Holiday），故手動修改
movie_name_list[5465] = "A Charlie Brown Christmas" # 片名錯誤（A Charlie Brown Christmas），故手動修改
movie_name_list[5467] = "Escape from Sobibor" # 片名錯誤（Escape from Sobibor），故手動修改
movie_name_list[5506] = "Peace, Propaganda & the Promised Land" # 片名錯誤（Peace, Propaganda & the Promised Land），故手動修改
movie_name_list[5571] = "Family Motocross" # 片名錯誤（Family Motocross），故手動修改    

In [8]:
#將電影名稱轉換為IMDB電影網（IMDB是透過ID查詢，所以要用查詢電影名的方式獲取正確的URL）
for i,x in enumerate(movie_name_list):
    title_for_url = urllib.parse.quote_plus(x.encode('utf-8')) #利用urllib轉換正確的URL
    imdb_search_link = "https://www.imdb.com/find?ref_=nv_sr_fn&q={}&s=tt&ref_=fn_tt".format(title_for_url)
    imdb_search_urls.append(imdb_search_link)
    #print("開始轉換IMDB電影網搜尋網址，第",i+1,"部，電影名稱： ", x) #檢視錯誤用
    #print(imdb_search_link) #檢視錯誤用
#print( imdb_search_urls[589]) #單個除錯用

In [14]:
imdb_urls = [] #存放該電影imdb完整的url
imdb_error_urls = {} #存放錯誤電影的url

for i,x in enumerate(imdb_search_urls):
    try:
        resp = requests.get(x) # GET request from url and parse via BeautifulSoup
        resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
        soup = BeautifulSoup(resp.text, 'lxml')
        # 因為已經過濾掉版面所以直接獲取第一順位的URL就是電影的IMDB頁面
        imdb_numbers_url = soup.find("td",{"class":"result_text"}).find_all("a")[0]["href"]
        # 前面加上imdb的網址變成完整的url
        imdb_url = "https://www.imdb.com{}".format(imdb_numbers_url)
        print("開始轉換IMDB電影網網址，第",i,"部") #檢測第幾部電影出錯
        print(imdb_url) #檢測第幾部電影的URL出錯
        imdb_urls.append(imdb_url)
        
    except AttributeError: #發生錯誤(屬性錯誤)時將錯誤的電影URL存ii放於list中
        error_element = x 
        error_number = i
        imdb_error_urls [error_number] = error_element;
        
print('---FINISH---')

開始轉換IMDB電影網網址，第 0 部
https://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 1 部
https://www.imdb.com/title/tt1298650/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 2 部
https://www.imdb.com/title/tt2395427/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 3 部
https://www.imdb.com/title/tt2488496/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 4 部
https://www.imdb.com/title/tt4154756/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5 部
https://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 6 部
https://www.imdb.com/title/tt0974015/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 7 部
https://www.imdb.com/title/tt2379713/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 8 部
https://www.imdb.com/title/tt1345836/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 9 部
https://www.imdb.com/title/tt1210819/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 10 部
https://www.imdb.com/title/tt0401729/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 11 部
https://www.imdb.com/title/tt0398286/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 12 部
https://www.imdb.com/title/tt0413300/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 13 部
https://www.im

開始轉換IMDB電影網網址，第 110 部
https://www.imdb.com/title/tt0338348/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 111 部
https://www.imdb.com/title/tt1735898/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 112 部
https://www.imdb.com/title/tt0181852/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 113 部
https://www.imdb.com/title/tt5197820/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 114 部
https://www.imdb.com/title/tt1964418/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 115 部
https://www.imdb.com/title/tt2567026/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 116 部
https://www.imdb.com/title/tt0892791/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 117 部
https://www.imdb.com/title/tt1211837/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 118 部
https://www.imdb.com/title/tt2245084/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 119 部
https://www.imdb.com/title/tt0892769/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 120 部
https://www.imdb.com/title/tt1772341/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 121 部
https://www.imdb.com/title/tt0816692/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 122 部
https://www.imdb.com/title/tt1628841/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網

開始轉換IMDB電影網網址，第 218 部
https://www.imdb.com/title/tt1860353/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 219 部
https://www.imdb.com/title/tt3949660/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 220 部
https://www.imdb.com/title/tt1402488/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 221 部
https://www.imdb.com/title/tt0086759/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 222 部
https://www.imdb.com/title/tt0407304/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 223 部
https://www.imdb.com/title/tt1911658/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 224 部
https://www.imdb.com/title/tt1951264/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 225 部
https://www.imdb.com/title/tt0304141/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 226 部
https://www.imdb.com/title/tt0440963/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 227 部
https://www.imdb.com/title/tt0441773/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 228 部
https://www.imdb.com/title/tt5095030/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 229 部
https://www.imdb.com/title/tt0478970/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 230 部
https://www.imdb.com/title/tt2069797/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網

開始轉換IMDB電影網網址，第 326 部
https://www.imdb.com/title/tt0312528/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 327 部
https://www.imdb.com/title/tt0248667/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 328 部
https://www.imdb.com/title/tt3659388/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 329 部
https://www.imdb.com/title/tt3640424/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 330 部
https://www.imdb.com/title/tt0343818/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 331 部
https://www.imdb.com/title/tt0164912/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 332 部
https://www.imdb.com/title/tt0780521/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 333 部
https://www.imdb.com/title/tt1869849/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 334 部
https://www.imdb.com/title/tt3416828/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 335 部
https://www.imdb.com/title/tt0141907/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 336 部
https://www.imdb.com/title/tt4649466/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 337 部
https://www.imdb.com/title/tt0187078/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 338 部
https://www.imdb.com/title/tt0172495/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網

開始轉換IMDB電影網網址，第 434 部
https://www.imdb.com/title/tt0230011/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 435 部
https://www.imdb.com/title/tt0985699/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 436 部
https://www.imdb.com/title/tt2120120/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 437 部
https://www.imdb.com/title/tt0212720/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 438 部
https://www.imdb.com/title/tt0338094/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 439 部
https://www.imdb.com/title/tt0164052/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 440 部
https://www.imdb.com/title/tt0373926/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 441 部
https://www.imdb.com/title/tt1854564/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 442 部
https://www.imdb.com/title/tt0325703/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 443 部
https://www.imdb.com/title/tt3110958/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 444 部
https://www.imdb.com/title/tt0266987/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 445 部
https://www.imdb.com/title/tt2569088/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 446 部
https://www.imdb.com/title/tt0183523/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網

...

開始轉換IMDB電影網網址，第 5147 部
https://www.imdb.com/title/tt1769363/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5148 部
https://www.imdb.com/title/tt1935902/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5149 部
https://www.imdb.com/title/tt3513086/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5150 部
https://www.imdb.com/title/tt1852001/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5151 部
https://www.imdb.com/title/tt0034492/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5152 部
https://www.imdb.com/title/tt1996264/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5153 部
https://www.imdb.com/title/tt0345551/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5154 部
https://www.imdb.com/title/tt1094666/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5155 部
https://www.imdb.com/title/tt2311428/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5156 部
https://www.imdb.com/title/tt0882977/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5157 部
https://www.imdb.com/title/tt0870118/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5158 部
https://www.imdb.com/title/tt1405365/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5159 部
https://www.imdb.com/title/tt0295725/?ref_=fn_tt_tt_1

開始轉換IMDB電影網網址，第 5254 部
https://www.imdb.com/title/tt0385056/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5255 部
https://www.imdb.com/title/tt2654360/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5256 部
https://www.imdb.com/title/tt1606829/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5257 部
https://www.imdb.com/title/tt3183630/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5258 部
https://www.imdb.com/title/tt1612774/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5259 部
https://www.imdb.com/title/tt0104567/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5260 部
https://www.imdb.com/title/tt0843850/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5261 部
https://www.imdb.com/title/tt0893346/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5262 部
https://www.imdb.com/title/tt4826674/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5263 部
https://www.imdb.com/title/tt2069797/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5264 部
https://www.imdb.com/title/tt0266971/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5265 部
https://www.imdb.com/title/tt0489212/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5266 部
https://www.imdb.com/title/tt0486541/?ref_=fn_tt_tt_1

開始轉換IMDB電影網網址，第 5361 部
https://www.imdb.com/title/tt0072856/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5362 部
https://www.imdb.com/title/tt1568341/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5363 部
https://www.imdb.com/title/tt0100146/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5364 部
https://www.imdb.com/title/tt0445922/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5365 部
https://www.imdb.com/title/tt1991199/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5366 部
https://www.imdb.com/title/tt3512332/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5367 部
https://www.imdb.com/title/tt0393049/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5368 部
https://www.imdb.com/title/tt3976258/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5369 部
https://www.imdb.com/title/tt1328873/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5370 部
https://www.imdb.com/title/tt1971352/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5371 部
https://www.imdb.com/title/tt0082467/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5372 部
https://www.imdb.com/title/tt0118842/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5373 部
https://www.imdb.com/title/tt0258273/?ref_=fn_tt_tt_1

開始轉換IMDB電影網網址，第 5468 部
https://www.imdb.com/title/tt0072271/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5469 部
https://www.imdb.com/title/tt0098213/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5470 部
https://www.imdb.com/title/tt1748207/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5471 部
https://www.imdb.com/title/tt0083722/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5472 部
https://www.imdb.com/title/tt2125430/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5473 部
https://www.imdb.com/title/tt2638024/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5474 部
https://www.imdb.com/title/tt4607906/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5475 部
https://www.imdb.com/title/tt1742336/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5476 部
https://www.imdb.com/title/tt0488873/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5477 部
https://www.imdb.com/title/tt0796314/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5478 部
https://www.imdb.com/title/tt0058701/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5479 部
https://www.imdb.com/title/tt0063350/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5480 部
https://www.imdb.com/title/tt0004972/?ref_=fn_tt_tt_1

開始轉換IMDB電影網網址，第 5575 部
https://www.imdb.com/title/tt0428303/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5576 部
https://www.imdb.com/title/tt0430371/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5577 部
https://www.imdb.com/title/tt2071645/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5578 部
https://www.imdb.com/title/tt0461832/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5579 部
https://www.imdb.com/title/tt2107644/?ref_=fn_tt_tt_1
開始轉換IMDB電影網網址，第 5580 部
https://www.imdb.com/title/tt0378407/?ref_=fn_tt_tt_1
---FINISH---


In [20]:
imdb_error_urls 

{}

In [19]:
imdb_movie_urls=DataFrame(imdb_urls,columns = ['imdb_url'])#將陣列轉換為dataframe
imdb_movie_urls.to_csv("imdb_movie_urls", index = False, encoding="utf-8")#儲存為csv檔
imdb_movie_urls.head()

,imdb_url
0,https://www.imdb.com/title/tt0499549/?ref_=fn_...
1,https://www.imdb.com/title/tt1298650/?ref_=fn_...
2,https://www.imdb.com/title/tt2395427/?ref_=fn_...
3,https://www.imdb.com/title/tt2488496/?ref_=fn_...
4,https://www.imdb.com/title/tt4154756/?ref_=fn_...


In [ ]:
'''
#單篇測試用
url = 'https://www.imdb.com/find?ref_=nv_sr_fn&q=Dungeons+%26+Dragons%3A+Wrath+of+the+Dragon+God&s=tt&ttype=ft&ref_=fn_ft' # define url for crawling
resp = requests.get(url) # GET request from url and parse via BeautifulSoup
resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
soup = BeautifulSoup(resp.text, 'lxml')
# 因為已經過濾掉版面所以直接獲取第一順位的URL就是電影的IMDB頁面
imdb_url = soup.find("td",{"class":"result_text"}).find_all("a")[0]["href"]
imdb_url
'''